## Escalamiento de los datos

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.pipeline import make_pipeline

from IPython.display import display


<div class="alert alert-success">
  Diversos algoritmos son sensibles a la escala en la que viene cada feature. **Re-escalarlos** puede traer significativas mejoras de rendimiento.
</div>

Existen distintas estrategias de escalamiento de tus features, pero **la más común es la estandarización** donde convertimos la variable para que la distribución de esta siga una distribución que es Gaussiana de media 0 y de desviación estandar 1.

In [2]:
X = pd.read_csv('./data/transformed/peliculas_clean.csv')
y = X['worldwide_gross']
X.drop(columns=['worldwide_gross', 'gross'],inplace=True)
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [3]:
scaler = StandardScaler()
scaler.fit(X_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [4]:
display(scaler.mean_)
display(scaler.scale_)

array([3.37799072e+07, 2.00217523e+03, 2.12518071e+00, 1.08645887e+02,
       1.02908129e+04, 4.10876582e+07, 6.46075374e+00])

array([4.19639260e+07, 1.20004440e+01, 7.19709390e-01, 2.32018033e+01,
       1.63686566e+04, 2.28348932e+08, 1.07571078e+00])

In [5]:
display(X.values)
display(scaler.transform(X_train))

array([[4.25000000e+08, 2.00900000e+03, 1.78000000e+00, ...,
        4.83400000e+03, 2.37000000e+08, 7.90000000e+00],
       [3.06000000e+08, 2.00213073e+03, 2.12697615e+00, ...,
        1.43000000e+02, 4.04553863e+07, 7.10000000e+00],
       [3.00000000e+08, 2.00700000e+03, 2.35000000e+00, ...,
        4.83500000e+04, 3.00000000e+08, 7.10000000e+00],
       ...,
       [7.00000000e+03, 2.00500000e+03, 2.12697615e+00, ...,
        9.30000000e+01, 3.25000000e+03, 7.80000000e+00],
       [3.96700000e+03, 2.01200000e+03, 2.35000000e+00, ...,
        2.38600000e+03, 4.04553863e+07, 6.30000000e+00],
       [1.10000000e+03, 2.00400000e+03, 1.85000000e+00, ...,
        1.63000000e+02, 1.10000000e+03, 6.60000000e+00]])

array([[-5.19015003e-01, -9.79320818e-02, -3.82349750e-01, ...,
        -3.50658761e-01, -1.27382502e-01, -1.07905745e+00],
       [ 1.95884743e-01, -7.64574084e-01, -3.82349750e-01, ...,
        -1.56873769e-01,  3.99538473e-03, -2.28756072e+00],
       [ 5.15252298e+00,  9.85361172e-01,  3.12375103e-01, ...,
        -6.95727750e-02,  9.14882063e-01,  9.66101931e-01],
       ...,
       [-4.14163040e-01, -1.09789509e+00, -3.82349750e-01, ...,
         2.52422590e+00, -1.08113745e-01,  1.43091088e+00],
       [-7.78761910e-01, -1.46018315e-02, -1.10486360e+00, ...,
        -6.28690129e-01, -2.76888457e-03, -1.63682819e+00],
       [-3.04545079e-01,  8.18700672e-01,  3.12375103e-01, ...,
         3.77698601e+00, -8.79691357e-02,  1.24498730e+00]])

In [6]:
X_train_scaled, X_test_scaled = (scaler.transform(X_train), scaler.transform(X_test))

In [11]:
model = Lasso()
model_scaled = Lasso()

model.fit(X_train,y_train)
model_scaled.fit(X_train_scaled,y_train)

print('{:.2%}'.format(model.score(X_test,y_test)))
print('{:.2%}'.format(model_scaled.score(X_test_scaled,y_test)))

52.63%
52.63%


<div class="alert alert-success">
  Los modelos de regresión no se ven afectados por el escalamiento de las features. Los de clasificación sí. 
</div>

## Simplificar las transformaciones con pipelines

<div class="alert alert-info">
  Para hacer tu código más reproducible, y para evitar tener que aplicar multiples veces una misma transformación te recomendamos utilizar <code> sklearn.pipeline.make_pipeline </code> que permite encadenar transformaciones a tus modelos.
</div>

In [10]:
model_scaled = make_pipeline(StandardScaler(),
                             Lasso() )
model_scaled.fit(X_train, y_train)
'{:.2%}'.format(model_scaled.score(X_test, y_test))

'52.63%'

## Crear nuevas features de forma automática

In [12]:
A = np.arange(6).reshape(3,2)
A

array([[0, 1],
       [2, 3],
       [4, 5]])

In [28]:
transformer.fit_transform(A)

array([[ 1.,  0.,  1.,  0.,  0.,  1.],
       [ 1.,  2.,  3.,  4.,  6.,  9.],
       [ 1.,  4.,  5., 16., 20., 25.]])

In [31]:
from sklearn.preprocessing import PolynomialFeatures

transformer = PolynomialFeatures(2)
A_transformed = transformer.fit_transform(A)

display(A.shape, A)
display(transformer.get_feature_names())
display(A_transformed.shape, A_transformed)

(3, 2)

array([[0, 1],
       [2, 3],
       [4, 5]])

['1', 'x0', 'x1', 'x0^2', 'x0 x1', 'x1^2']

(3, 6)

array([[ 1.,  0.,  1.,  0.,  0.,  1.],
       [ 1.,  2.,  3.,  4.,  6.,  9.],
       [ 1.,  4.,  5., 16., 20., 25.]])

``PolynomialFeatures`` transforma una matriz $(A1,A2)$ a $(1,A1,A2,A1^2,A1\cdot A2,A2^2)$

In [33]:
X.shape, transformer.fit_transform(X).shape

((4104, 7), (4104, 36))

In [35]:
model_poly = make_pipeline( PolynomialFeatures(2),
                            Lasso())
model_poly.fit(X_train, y_train)
'{:.2%}'.format(model_poly.score(X_test, y_test))

'57.38%'

## Crear features categóricas

<div class="alert alert-success">
  En terminos de Machine Learning a las features que pueden tomar un número finito de valores se les llama categóricas. Ejemplos para esto són: género, páis, grado académico, etc.
</div>

Un mapeo del tipo $\{Perú, Chile, Colombia, Venezuela\} \rightarrow \{1, 2, 3, 4\}$ tiene el problema de asignarle un ordén a los valores posibles de la categoría. Este orden impacta de distintas maneras los algoritmos de Machine Learning, por ejemplo aquellos que dependen de la topología de $R^n$ y de la función de distancia entre puntos en este espacio, considerarán que ciertas categorías se encuentran más cercanas unas de otras, siendo que esto es generado puramente artificialmente por el encoding, y no por las datos per se. 

Para no introducir información falsa o erronéa en nuestro modelos existen formas más inteligentes de encodear nuestros datos.

_**Encoding one-hot**_

Este encoding consiste en asignarle una columna a cada categoría y rellenarla con 0 y 1 de la forma siguiente:

In [36]:
d = pd.DataFrame([['Chile','Colombia','Colombia','Venezuela'],['hombre','mujer','hombre','mujer']])
d = d.T
d.columns = pd.Index(['pais','genero'])
d

,pais,genero
0,Chile,hombre
1,Colombia,mujer
2,Colombia,hombre
3,Venezuela,mujer


In [37]:
pd.get_dummies(d)

,pais_Chile,pais_Colombia,pais_Venezuela,genero_hombre,genero_mujer
0,1,0,0,1,0
1,0,1,0,0,1
2,0,1,0,1,0
3,0,0,1,0,1


Sklearn también ofrece un objeto OneHotEncoder pero es un poco más díficil de utilizar, así que por criterios pedagogicos hemos elegido ``pd.get_dummies``. Sin embargo el objeto de sklearn tiene la ventaja de ser pipeable, por lo que es bueno considerarlo para ciertos casos de uso.

Cuantas columnas generaríamos con un one-hot encoding de nuestras features categóricas?

In [41]:
movies_obj = pd.read_csv('./data/transformed/peliculas_obj.csv')
movies_obj.apply(pd.Series.nunique).sort_values()

color                2
content_rating      17
language            39
country             55
genres             809
actor_1_name      1704
director_name     1967
actor_2_name      2508
actor_3_name      2917
plot_keywords     3992
movie_title       4061
dtype: int64

Las features más informativas son las del casting. Si embargo haciendo un one-hot encoding de estas estaríamos aumentando la dimensión por 2000 y algo!!

## Encoding Binario

Esta técnica no es canónica por lo que tendremos que buscarla en otra librería. Sin embargo el autor tuvo la buena idea de hacer su API compatible con la de sklearn, así que no tendremos ninguna dificultad en usarla.

$$ Categoria \rightarrow Numero \rightarrow Binario \rightarrow Columnas $$

In [42]:
categoricals = movies_obj[['actor_1_name', 'director_name']]
categoricals.shape, categoricals.head(2)

((4104, 2),
   actor_1_name  director_name
 0  CCH Pounder  James Cameron
 1  Doug Walker    Doug Walker)

In [44]:
bin_encoder = pd.concat([X, categoricals], axis=1)
display(bin_encoder.shape)
bin_encoder.head(2)

,production_budget,title_year,aspect_ratio,duration,cast_total_facebook_likes,budget,imdb_score,actor_1_name,director_name
0,425000000,2009.000000,1.780000,178.000000,4834,2.370000e+08,7.9,CCH Pounder,James Cameron
1,306000000,2002.130733,2.126976,108.577186,143,4.045539e+07,7.1,Doug Walker,Doug Walker


In [47]:
import category_encoders as ce
encoder = ce.BinaryEncoder(cols=['actor_1_name', 'director_name'])
encoder

BinaryEncoder(cols=['actor_1_name', 'director_name'], drop_invariant=False,
              handle_missing='value', handle_unknown='value', mapping=None,
              return_df=True, verbose=0)

In [48]:
bin_encoder = encoder.fit_transform(bin_encoder)
bin_encoder.shape

(4104, 31)

In [49]:
Xb_train, Xb_test, y_train, y_test = train_test_split(bin_encoder, y)
X_train, X_test = (Xb_train[X.columns],Xb_test[X.columns])

model_binenc = Lasso()
model = Lasso()

model_binenc.fit(Xb_train, y_train)
model.fit(X_train, y_train)

print('{:.2%}'.format(model_binenc.score(Xb_test, y_test)))
print('{:.2%}'.format(model.score(X_test, y_test)))

57.10%
56.76%


Aumentamos el rendimiento de nuestro algoritmo pero no de forma significativa. Mantengamos entonces la dimensionalidad de nuestro espacio de features baja, y vamos a buscar modelos más complejos.

## Conocimiento experto

<div class="alert alert-success">
  Una grán parte del diseño de las features pasa por un **conocimiento espécifico del dominio en el que se esta trabajando**. <br>
  Por ejemplo para analizar una imagen nuestro cerebro no se concentra en los millones de pixeles de una imagen, pero sólo en algunos relevantes como los de los contornos. Durante un buen tiempo **los sistemas de visión de computadores encodeaban features que traducían este conocimiento experto (contornos).** <br>
  Una de las únicas formas de obtener este conocimiento de forma sistemática es ir a bucear en repositorios de papers de Machine Learning como Arxiv, y estudiar la investigación que se ha hecho sobre el dominio específico.
</div>

## Más datos de calidad

Nada le gana conseguir más datos que sean encodeables en features de calidad.

_**Píramide de Maslow del Machine Learning**_

<img src="./img/maslow.png">

Contamos con la base de datos de ganancias de las péliculas el primer fin de semana de exhibición, así como la cantidad de cines en la que fue estrenada.

Puedes mejorar considerablemente nuestra predicción?